In [ ]:
import sys  
sys.path.insert(0, r'F:\raster4ml')

In [ ]:
import os
import folium
import numpy as np
import numpy.ma as ma
import rasterio
from raster4ml.preprocessing import reproject_raster
from pyproj import Transformer
import matplotlib.pyplot as plt

In [ ]:
img_dir = r'F:\raster4ml\data\landsat\LC08_L1TP_137045_20210317_20210328_01_T1\Stack.tif'
bands = [3, 2, 1]

In [ ]:
src = rasterio.open(img_dir)

In [ ]:
src_crs = src.crs.to_epsg() # Get src crs
dst_crs = 4326 # Destination crs (WGS 1984)

# Projection Convert
proj = Transformer.from_crs(src_crs, dst_crs, always_xy=True)
min_lon, min_lat, max_lon, max_lat = src.bounds # Get bounds
bound = [[min_lat, min_lon], [max_lat, max_lon]]
bound_wgs = []
for lat, lon in bound:
    proj = Transformer.from_crs(src_crs, dst_crs, always_xy=True)
    lon_x, lat_x = proj.transform(lon, lat)
    bound_wgs.append([lat_x, lon_x])
cen_lat, cen_lon = np.mean(np.array(bound_wgs), axis=0) # Center lat lon
    


In [ ]:
img = src.read()
img = np.moveaxis(img, 0, 2)
img = img[:, :, bands]

In [ ]:
img = img.astype(np.float32)
img[img==0.0] = np.nan

In [ ]:
img = ma.masked_invalid(img)

In [ ]:
img_norm = (img - img.min()) / (img.max() - img.min())
img_norm = ma.filled(img_norm, fill_value=0.0)

In [ ]:
mask = ma.getmask(img)
mask = mask * 1
mask = mask[:, :, 0]
mask = np.abs(mask-1)

In [ ]:
img_rgba = np.zeros(shape=(img.shape[0], img.shape[1], 4))

In [ ]:
img_rgba[:, :, :3] = img_norm

In [ ]:
img_rgba[:, :, 3] = mask

In [ ]:
"""for i in range(4):
    plt.imshow(img_cmap[:, :, i])
    plt.colorbar()
    plt.show()"""

In [ ]:
def colorize(array, cmap='viridis'):
    normed_data = (array - array.min()) / (array.max() - array.min())    
    cm = plt.cm.get_cmap(cmap)    
    return cm(normed_data)

In [ ]:
m = folium.Map(location=[cen_lat, cen_lon],
               zoom_start=7)

In [ ]:
folium.raster_layers.ImageOverlay(
    name='Image',
    image=img_rgba*20.0,
    bounds=bound_wgs,
    interactive=True,
).add_to(m)
folium.LayerControl().add_to(m)

In [ ]:
bound_wgs

In [ ]:
new_bound = [[25.580501113303256, 93.98474797298768],
 [27.75013948054664, 96.17508389240355]]

In [ ]:
m.fit_bounds(new_bound)

In [ ]:
m